In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from lifelines import CoxPHFitter, WeibullAFTFitter, KaplanMeierFitter
from sksurv.metrics import concordance_index_ipcw, concordance_index_censored
from sksurv.util import Surv
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression
from typing import Optional, Dict, Any


# 1) Comprendre les données et les objectifs

**Entrées** :  
- `clinical_train.csv` : Caractéristiques cliniques (BM_BLAST, WBC, ANC, MONOCYTES, HB, PLT, …) + ID.  
- `molecular_train.csv` : Mutations géniques (GENE, EFFECT, VAF, …) + ID (plusieurs lignes par patient).  
- `target_train.csv` : Labels de survie (OS_YEARS, OS_STATUS) + ID.  

**Sorties** : Modèle de prédiction de risque (CoxPH, Weibull AFT) ; C-index ; stratification KM par décile ; calibration de la probabilité de survie à des points temporels (1 an, 2 ans, 3 ans).

## 1. `clinical_train.csv` — Données cliniques
Chaque ligne = 1 patient au diagnostic, contenant des informations hématologiques, centre, cytogénétique.

| Colonne | Signification | Rôle |
|---------|---------------|------|
| **ID** | Code patient unique | Fusion des tables ; ne pas utiliser comme feature. |
| **CENTER** | Centre médical (MSK, DFCI, …) | Vérifier le biais ; strata si nécessaire. |
| **BM_BLAST** | % blast de moelle osseuse | ↑ → pronostic défavorable (invasion élevée). |
| **WBC** | Globules blancs périphériques (×10⁹/L) | ↑ → charge de maladie, complications. |
| **ANC** | Neutrophiles absolus | ↓ → infection, défavorable. |
| **MONOCYTES** | Monocytes | Lié à AML M4/M5. |
| **HB** | Hémoglobine (g/dL) | ↓ → anémie, défavorable. |
| **PLT** | Plaquettes (×10⁹/L) | ↓ → hémorragie, défavorable. |
| **CYTOGENETICS** | Chromosomes (ex. : del(20), t(3;9)) | Extraire risque ELN (anomalies 3q, 5q, 7q). |

**Exemple** :
| ID | CENTER | BM_BLAST | WBC | ANC | MONOCYTES | HB | PLT | CYTOGENETICS |
|----|--------|----------|-----|-----|-----------|----|-----|--------------|
| P132697 | MSK | 14.0 | 2.8 | 0.2 | 0.7 | 7.6 | 119 | 46,xy,del(20)(q12)[2]/46,xy[18] |
| P132700 | MSK | 6.0 | 128 | 9.7 | 0.9 | 11.1 | 195 | 46,xx,t(3;9)(p13;q22)[10]/46,xx[10] |

## 2. `molecular_train.csv` — Données de mutations géniques
Chaque ligne = 1 mutation par patient (niveau variant).

| Colonne | Signification | Utilisation |
|---------|---------------|-------------|
| **ID** | Code patient | Groupby pour synthétiser. |
| **CHR/START/END** | Position chromosomique | Non nécessaire (sauf feature génomique). |
| **REF/ALT** | Allèle de référence/alternative | Référence ; ne pas entraîner. |
| **GENE** | Gène muté (TP53, NPM1, …) | Créer binaire (oui/non). |
| **PROTEIN_CHANGE** | Changement protéique (p.R1262L) | Classer missense/truncating. |
| **EFFECT** | Type (missense, frameshift, …) | Grouper délétère/neutre. |
| **VAF** | % allèle muté | Quantitatif (taille clone). |
| **DEPTH** | Profondeur de lecture | Vérifier qualité. |

**Exemple** :
| ID | GENE | EFFECT | VAF | DEPTH |
|----|------|--------|-----|-------|
| P100000 | CBL | non_synonymous_codon | 0.083 | 1308 |
| P100000 | DNMT3A | frameshift_variant | 0.0898 | 942 |
| P100000 | TET2 | non_synonymous_codon | 0.43 | 826 |

### Synthèse par patient (groupby ID)
| Caractéristique | Calcul | Signification |
|-----------------|--------|---------------|
| `n_genes` | Nombre de gènes mutés uniques | Complexité génétique. |
| `vaf_mean` | Moyenne VAF | Clone moyen. |
| `vaf_max` | VAF maximum | Clone majoritaire. |
| `TP53_mut`, `NPM1_mut`, … | 1 si mutation présente | Stratification pronostic. |

## 3. `target_train.csv` — Données de survie
| Colonne | Signification | Rôle |
|---------|---------------|------|
| **ID** | Code patient | Fusion. |
| **OS_YEARS** | Temps de survie/suivi (années) | Duration pour modèle survie. |
| **OS_STATUS** | 1=décès ; 0=censure | Event pour modèle survie. |

## 4. Signification médicale
| Variable | Type | Signification clinique | Impact |
|----------|------|------------------------|--------|
| **BM_BLAST** | Clinique | Invasion moelle | ↑ → défavorable |
| **WBC** | Clinique | Charge de maladie | ↑ → défavorable |
| **HB** | Clinique | Anémie | ↓ → défavorable |
| **PLT** | Clinique | Hémorragie | ↓ → défavorable |
| **n_genes** | Moléculaire | Complexité génétique | ↑ → défavorable |
| **vaf_max** | Moléculaire | Clone principal | ↑ → défavorable |
| **TP53_mut** | Moléculaire | Malignité élevée | ↑ → défavorable |
| **NPM1_mut**, **CEBPA_mut**, **IDH1/2_mut** | Moléculaire | Favorable | ↓ → favorable |

In [ ]:
clinical = pd.read_csv("./data/clinical_train.csv")
molecular = pd.read_csv("./data/molecular_train.csv")
target = pd.read_csv("./data/target_train.csv")


In [ ]:
clinical.head()

In [ ]:
molecular.head()

In [ ]:
target.head()

# 2) Normalisation des étiquettes et fusion des données

* Fusionner `clinical` avec `target` par `ID`.
* À partir de `molecular`, synthétiser par **patient** (par `ID`) :

  * Nombre de gènes différents (`n_genes`), moyenne/max VAF (`vaf_mean`, `vaf_max`).
  * (Optionnel) Compter les variants par EFFECT (missense/stop_gained/frameshift/splice…), et one-hot encoder les gènes AML importants (NPM1, FLT3, TP53, DNMT3A, IDH1/2, RUNX1, TET2, NRAS, KRAS, CEBPA).
* Fusionner les caractéristiques moléculaires synthétisées dans le tableau clinique-objectif → créer **tableau de travail**.

In [ ]:

t = target.copy()
# t["OS_STATUS_BIN"] = t["OS_STATUS"].fillna(0)

clin = clinical.merge(t[["ID","OS_YEARS","OS_STATUS"]], on="ID", how="left")

# Tổng hợp phân tử
agg_f = molecular.groupby("ID").agg(
    n_genes=("GENE","nunique"),
    vaf_mean=("VAF","mean"),
    vaf_max=("VAF","max")
).reset_index()
agg_f


In [ ]:
# Gộp
full = clin.merge(agg_f, on="ID", how="left")
full = full.fillna(0)
full


# 3) Nettoyage et prétraitement des caractéristiques

1. **Valeurs manquantes et infinies**

   * Remplacer `±inf` par manquant ; imputer les manquants pour les variables explicatives (suggestion : médiane) mais **ne pas imputer** pour `OS_YEARS`, `OS_STATUS_BIN`.

2. **Temps non positif**

   * Pour AFT, exiger `OS_YEARS > 0`. Si valeurs ≤ 0, ajouter un epsilon très petit pour assurer positivité.

3. **Gestion de la skewness et des outliers**

   * Variables très skewées (WBC, ANC, PLT, VAF…) devraient utiliser **log1p** ou transformation similaire.

   * **Winsorize**/clip aux percentiles fins (ex. 0.5%–99.5%) pour réduire l'impact des outliers extrêmes.

4. **Supprimer variables peu informatives**

   * Supprimer variables avec **variance proche de 0** ou **presque une seule valeur** (bruit).

5. **Réduire la multicolinéarité**

   * Vérifier matrice de corrélation ; supprimer l'une des deux variables avec |ρ| trop élevé (ex. > 0.95).

   * (Optionnel) Calculer **VIF**, supprimer progressivement variables avec VIF trop grand (ex. > 10).

6. **Normalisation des échelles**

   * Scaler (z-score) toutes les caractéristiques d'entrée pour stabiliser l'optimisation, faciliter comparaison des coefficients.

In [ ]:

duration_col = "OS_YEARS"
event_col    = "OS_STATUS"
features     = ["BM_BLAST","WBC","ANC","MONOCYTES","HB","PLT","n_genes","vaf_mean","vaf_max"]

In [ ]:

df = full[[duration_col, event_col] + features].copy()
df.replace([np.inf, -np.inf], np.nan, inplace=True)



In [ ]:

# Remplir les NaN pour X, assurer la présence de y
for c in features:
    df[c] = df[c].fillna(df[c].median())
df = df.dropna(subset=[duration_col, event_col])


In [ ]:

# Si temps ≤ 0, ajuster à epsilon pour AFT
epsilon = 1e-6
df.loc[df[duration_col] <= 0, duration_col] = epsilon

In [ ]:
for c in ["WBC","ANC","PLT","vaf_mean","vaf_max"]:
    if c in df.columns:
        df[c] = np.log1p(df[c])

low_q, high_q = 0.005, 0.995
for c in features:
    lo, hi = df[c].quantile(low_q), df[c].quantile(high_q)
    df[c] = df[c].clip(lo, hi)


In [ ]:

# drop near-zero variance
nzv = [c for c in features if (df[c].std() < 1e-8 or df[c].nunique() <= 2)]
features = [c for c in features if c not in nzv]


In [ ]:

# drop high corr
corr = df[features].corr().abs()
upper = corr.where(np.triu(np.ones_like(corr), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]
features = [c for c in features if c not in to_drop]

In [ ]:
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

print("\nCaractéristiques conservées :", features)
print("Taux d'événements :", df[event_col].mean())

# 5) Division de l'ensemble de données en garantissant la présence d'événements

**Problème** : Les données de survie sont facilement déséquilibrées lors d'une division aléatoire, menant à un ensemble de test sans événements (N₁^(test) = 0).

**Traitement** : Stratification par événement (δ_i) pour maintenir un taux de mortalité r = N₁/N équivalent entre train/test :
$$
\frac{N_1^{(train)}}{N^{(train)}} \approx \frac{N_1^{(test)}}{N^{(test)}} \approx r.
$$

Algorithme de stratification :
1. Diviser les données : D₁ = {i : δ_i=1}, D₀ = {i : δ_i=0}.
2. Échantillonner aléatoirement un ratio p de chaque groupe : D₁^(train) = sample(D₁, p), D₀^(train) = sample(D₀, p).
3. Fusionner : D^(train) = D₁^(train) ∪ D₀^(train) ; D^(test) = D \ D^(train).

→ Assurer une distribution d'événements similaire.

**Contraintes** : Train/test ≥ N_min événements (N_min ≈ 5–10). Si manque dans test : réduire test_size ou transférer quelques événements de train vers test.

**Avec données petites/r rares** (N₁^(test/train) < N_min) :  
- (a) Réduire test_size (ex. : 0.3 → 0.2).  
- (b) Transférer aléatoirement des événements de train vers test jusqu'à suffisance.  

→ Assurer la validité pour les modèles Cox/AFT.

In [ ]:

def stratified_surv_split(df, event_col="OS_STATUS",
                          test_size=0.25, min_events_test=5,
                          max_tries=80, random_state=123):
    y = df[event_col].astype(int).values
    ts = test_size
    for i in range(max_tries):
        tr, te = train_test_split(df, test_size=ts, random_state=random_state+i, stratify=y)
        if tr[event_col].sum() >= min_events_test and te[event_col].sum() >= min_events_test:
            return tr, te
        ts = max(0.10, ts - 0.05)
    return tr, te  # fallback

train_df, test_df = stratified_surv_split(df, event_col=event_col, test_size=0.25, min_events_test=5)
print("events train/test:", int(train_df[event_col].sum()), int(test_df[event_col].sum()))

# nếu vẫn đen đủi, ép chuyển 1 ít event sang test:
if test_df[event_col].sum() == 0 and train_df[event_col].sum() > 0:
    pos_idx = train_df[train_df[event_col]==1].sample(n=min(5, int(train_df[event_col].sum())), random_state=7).index
    move_rows = train_df.loc[pos_idx]
    train_df = train_df.drop(index=pos_idx)
    test_df  = pd.concat([test_df, move_rows], axis=0)

print("events train/test (after fix):", int(train_df[event_col].sum()), int(test_df[event_col].sum()))



# 6) Modelling


In [ ]:
Xcols        = features  # đã chuẩn bị ở các bước trước

--------------------

## 6.1) Fit models 

### 6.1.1) Modèle des Hasards Proportionnels de Cox (CoxPH)

**Description courte** : Ajouter un **pénaliseur ridge** faible (L2) pour stabiliser en cas de multicolinéarité/peu d'événements. Utiliser le **hasard partiel** comme mesure de score de risque.

Le CoxPH est le modèle central pour l'analyse de survie, estimant l'impact des caractéristiques (X) sur le taux de hasard au fil du temps.

Mục tiêu

Hasard pour l'individu i :
$$
h_i(t \mid X_i) = h_0(t) \exp(\beta^\top X_i)
$$
- $h_0(t)$ : hasard de base.
- $\beta$ : coefficients à apprendre.
- Hypothèse : hasard proportionnel fixe entre les individus (proportional hazards).

Partial Likelihood

Estimer $\beta$ par partial likelihood (ignorant $h_0(t)$) :
$$
L(\beta) = \prod_{i:\delta_i=1} \frac{\exp(\beta^\top X_i)}{\sum_{j \in R(T_i)} \exp(\beta^\top X_j)}
$$
Log-likelihood :
$$
\ell(\beta) = \sum_{i:\delta_i=1} \left[ \beta^\top X_i - \log \sum_{j \in R(T_i)} \exp(\beta^\top X_j) \right]
$$
Maximiser $\ell(\beta)$ pour obtenir $\hat{\beta}$.

Vấn đề

Avec p variables ≈ N₁ événements ou multicolinéarité (ex. : WBC ~ BM_BLAST), la matrice hessienne $H = X^\top W X$ est presque singulière → non convergence.

Ridge Penalization (L2)

Ajouter une pénalité :
$$
\ell_{ridge}(\beta) = \ell(\beta) - \frac{\lambda}{2} \|\beta\|_2^2 \quad (\lambda \in [0.01, 0.5])
$$
→ Stabiliser $\hat{\beta}$, réduire l'overfitting, lisser les poids.

Partial Hazard (Risk Score)

$$
\text{partial hazard}_i = \exp(\hat{\beta}^\top X_i), \quad \text{risk score}_i = \hat{\beta}^\top X_i
$$
- Comparaison : Si risk score_i > risk score_j → i a un risque plus élevé que j.
- HR entre i et j : $\exp(\hat{\beta}^\top (X_i - X_j))$.
- Applications : Groupement (Kaplan–Meier), C-index, graphique de calibration.

Quy trình thực hành

| Étape | Objectif | Réalisation |
|-------|----------|-------------|
| 1. Z-score X | Éviter que les variables grandes dominent | $X_j' = \frac{X_j - \mu_j}{\sigma_j}$ |
| 2. Ridge faible | Stabiliser | `CoxPHFitter(penalizer=0.1, robust=True)` |
| 3. Vérifier convergence | Éviter singulier | Réduire variables/augmenter λ si nécessaire |
| 4. Prédire | Extraire risque | `cph.predict_partial_hazard(X_test)` |

Exemple de résultats

| Variable | $\hat{\beta}$ | HR = exp($\hat{\beta}$) | Interprétation |
|----------|-----------------|---------------------------|----------------|
| BM_BLAST | +0.031 | 1.03 | +1% blast → hasard ↑3% |
| WBC | +0.008 | 1.01 | WBC ↑ → risque ↑ léger |
| PLT | -0.004 | 0.996 | Plaquettes hautes → risque ↓ |
| TP53_mut | +0.85 | 2.34 | Mutation présente → risque ×2.3 |

Tổng kết

| Composant | Expression | Signification |
|-----------|------------|---------------|
| Hasard | $h_i(t) = h_0(t) e^{\beta^\top X_i}$ | Vitesse de mortalité instantanée |
| Log partial-LL | $\sum [\beta^\top X_i - \log \sum e^{\beta^\top X_j}]$ | Estimation de $\beta$ |
| Pénalité ridge | $-\frac{\lambda}{2}\|\beta\|_2^2$ | Stabiliser multicolinéarité |
| HR | $e^{\beta^\top(X_i - X_j)}$ | Comparer risques |
| Hasard partiel | $\exp(\beta^\top X_i)$ | Score de risque pour classement |

**Conclusion** : Le CoxPH estime l'impact relatif de X sur le risque. Le ridge faible stabilise en cas de peu d'événements/multicolinéarité. Le hasard partiel sert de score de risque pour la stratification, courbe KM, C-index/calibration.

In [ ]:

# Cox (ridge pour eviter singular)
cph = CoxPHFitter(penalizer=0.1)
cph.fit(train_df[[duration_col, event_col] + Xcols],
        duration_col=duration_col, event_col=event_col, robust=True)
risk_cox = cph.predict_partial_hazard(test_df[Xcols]).values.ravel()
risk_cox

### 6.1.2) Modèle du Temps d'Échec Accéléré de Weibull (AFT)

**Description courte** : Adapté quand CoxPH converge mal ou hypothèse PH non satisfaite. Utiliser **temps de survie médian prédit** ; inverser le signe pour obtenir “score de risque” (temps court → risque élevé).

Le Weibull AFT est un modèle paramétrique alternatif à Cox, modélisant directement le temps de survie (T) au lieu du hasard relatif.

Objectif

Log du temps de survie :
$$
\log(T_i) = \beta^\top X_i + \sigma \epsilon_i
$$
- $X_i$ : caractéristiques ; $\beta$ : coefficients ; $\sigma > 0$ : échelle ; $\epsilon_i$ : bruit (Gumbel pour Weibull).

Fonction de survie & Hasard

Fonction de survie :
$$
S(t \mid X) = \exp\left[-\left(\frac{t}{\lambda(X)}\right)^{\kappa}\right], \quad \lambda(X) = \exp(\beta^\top X), \quad \kappa = 1/\sigma
$$
Hasard :
$$
h(t \mid X) = \frac{\kappa}{\lambda(X)} \left(\frac{t}{\lambda(X)}\right)^{\kappa - 1}
$$
- $\kappa = 1$ : hasard constant ; $\kappa > 1$ : croissant ; $\kappa < 1$ : décroissant.

Estimation

Maximum de vraisemblance sur données censurées :
$$
\ell(\beta, \sigma) = \sum_i \left[ \delta_i \log f(T_i \mid X_i) + (1-\delta_i) \log S(T_i \mid X_i) \right]
$$
Implémentation : `WeibullAFTFitter().fit(df, duration_col="OS_YEARS", event_col="OS_STATUS_BIN")` (lifelines).

Médiane de survie

$$
t_{0.5}(X) = \lambda(X) (\ln 2)^{1/\kappa} = e^{\beta^\top X} (\ln 2)^{1/\kappa}
$$
Prédiction : `aft.predict_median(X_test)`.

Score de risque

Inverser le signe pour risque élevé → survie courte :
$$
\text{risk}_i = -t_{0.5}(X_i) \propto -\exp(\beta^\top X_i)
$$

Quand utiliser AFT au lieu de Cox

| Situation | Raison | Avantage AFT |
|-----------|--------|--------------|
| Cox ne converge pas | Hessienne singulière, peu d'événements | Paramétrique stable |
| Violation PH | HR change avec le temps | Pas besoin de PH |
| Besoin de temps absolu | Cox seulement relatif | Prédiction T directe |
| Données petites | Variables fortement corrélées | Exploite forme paramétrique |

Procédure pratique

| Étape | Objectif | Réalisation |
|-------|----------|-------------|
| 1. Normaliser X | Convergence facile | Z-score/transformation log |
| 2. Choisir Weibull | Distribution appropriée | `WeibullAFTFitter()` |
| 3. Ajuster ML | Estimer $\beta, \sigma$ | `aft.fit(...)` |
| 4. Prédire médiane | Prédiction personnalisée | `aft.predict_median(X_test)` |
| 5. Inverser signe risque | Comparer avec Cox | `risk = -predict_median` |
| 6. Évaluer | Stratification | C-index/graphique KM |

Comparaison CoxPH vs. Weibull AFT

| Caractéristique | **CoxPH** | **Weibull AFT** |
|-----------------|-----------|-----------------|
| Objectif | Hasard relatif | Temps absolu |
| Hasard | $h_0(t) e^{\beta^\top X}$ | $\frac{\kappa}{\lambda} (t/\lambda)^{\kappa-1}$ |
| Hypothèse PH | Oui | Non obligatoire |
| Base | Non définie | Paramétrique Weibull |
| Score de risque | $\exp(\beta^\top X)$ | $-t_{0.5} = -e^{\beta^\top X} (\ln 2)^{1/\kappa}$ |
| Prédiction T absolue | Non | Oui (médiane/moyenne) |
| Stabilité données petites | Moyenne | Meilleure |

Signification pratique (AML)

- $\hat{t}_{0.5,i} < 2$ ans : high risk ; >5 : low risk.
- Utiliser –médiane comme risque pour KM, C-index, calibration.

**Conclusion** : Le Weibull AFT paramétrique modélise T directement, stable quand Cox converge mal/PH violée. Estime médiane de survie, inverse en score de risque pour stratification, évaluation similaire à Cox.

In [ ]:
# Weibull AFT
aft = WeibullAFTFitter()
aft.fit(train_df[[duration_col, event_col] + Xcols],
        duration_col=duration_col, event_col=event_col)
# median time nhỏ -> rủi ro lớn
risk_aft = -aft.predict_median(test_df[Xcols]).values.ravel()
risk_aft

# 3) Vérification de l'hypothèse PH (après ajustement de Cox)

**Description courte** : Utiliser le test/graphique de Schoenfeld pour identifier les variables violant PH ; violation grave → envisager **stratification** (couche par variable) ou **effets variant dans le temps**.

La vérification de l'hypothèse des hasards proportionnels (PH) est une étape obligatoire après l'ajustement de CoxPH, assurant un ratio de hasard fixe dans le temps.

## Contexte & Signification
CoxPH suppose :
$$
h(t \mid X) = h_0(t) \exp(\beta^\top X) \implies \frac{h(t \mid X_i)}{h(t \mid X_j)} = \exp[\beta^\top (X_i - X_j)]
$$
HR indépendant de t. Violation → $\beta$ biaisé.

## Quand violation PH
| Variable | Signe | Raison clinique |
|----------|-------|-----------------|
| WBC | Hasard élevé au début, décroissant | Décès précoce ; survivants postérieurs → pronostic normal |
| TP53_mut | Impact fort au début, décroissant | Traitement inverse le risque |
| CENTER | Stratégie change avec t | Violation par centre |

## Test des Résidus de Schoenfeld
Résidus :
$$
r_{ij} = x_{ij} - \frac{\sum_{k \in R(T_i)} x_{kj} e^{\beta^\top x_k}}{\sum_{k \in R(T_i)} e^{\beta^\top x_k}}
$$
- Si PH correct : $r_{ij}$ aléatoire autour de 0, indépendant de t.
- Violation : $r_{ij}$ tendance avec t.

## Test de Grambsch–Therneau
Test : $H_0: Cov(r_{ij}, \log t_i) = 0$.
p < 0.05 → violation. Global : $\chi^2_{\text{global}} = \sum_j \chi^2_j$.

Implémentation (lifelines) :
```python
from lifelines.statistics import proportional_hazard_test
results = proportional_hazard_test(cph, train_df, time_transform='rank')
results.print_summary(decimals=3)
```

Exemple :
| Variable | $\chi^2$ | p | Violation ? |
|----------|------------|---|-------------|
| BM_BLAST | 1.22 | 0.27 | Non |
| WBC | 5.45 | 0.019 | ✅ Oui |
| PLT | 0.31 | 0.58 | Non |
| TP53_mut | 4.92 | 0.027 | ✅ Oui |
| **Global** | 8.77 | 0.031 | ⚠️ Légère |

## Graphique de vérification
```python
cph.check_assumptions(train_df, show_plots=True)
```
- X : log(t) ; Y : résidus.
- Ligne verte plate → OK ; inclinée → violation (ex. : WBC descendant).

## Gestion des violations
### 7.1 Stratification
Variable catégorielle (ex. : CENTER) :
$$
h(t \mid X, \text{strata}) = h_{0,\text{strata}}(t) \exp(\beta^\top X)
$$
```python
cph.fit(df, ..., strata=["CENTER"])
```

### 7.2 Effets variant dans le temps
Variable quantitative (ex. : WBC) :
$$
\beta_j(t) = \beta_j + \gamma_j \log t
$$
```python
df["WBC_logt"] = df["WBC"] * np.log(df["OS_YEARS"])
cph.fit(df, ...)
```

### 7.3 Cox par tranches
Diviser intervalles temporels (0–2 ans, 2–5 ans, >5 ans) ; ajuster séparément par phase.

Résultats d'interprétation
| Variable | p-value | Violation ? | Mesure |
|----------|---------|-------------|--------|
| BM_BLAST | 0.27 | ❌ Non | Garder |
| WBC | 0.019 | ✅ Oui | Interaction log(t) |
| TP53_mut | 0.027 | ✅ Oui | Stratifier/time-varying |
| Global | 0.031 | ⚠️ Légère | Vérifier plus |

**Conclusion** : Vérification PH via test/graphique de Schoenfeld assure fiabilité du modèle. Violation légère : noter ; grave : stratifier/time-varying pour corriger, préserver sens HR fixe ou ajusté.

# 7) Évaluation de la discrimination : C-index (priorité à Uno’s C)

**Description courte** :  
1. **Uno’s C** (IPCW) : Mesure la discrimination en cas de censure ; nécessite une cohorte de référence pour estimer la probabilité de censure. Si train insuffisant → utiliser test comme référence.  
2. **Fallback Harrell’s C** : Si test sans événements ou erreur IPCW (peu d'échantillons après filtrage) → utiliser Harrell’s C comme mesure de secours.  
3. **Nettoyage des entrées** : Supprimer NaN/Inf dans temps/événements/risque ; si risque sans variance → C-index peu significatif, noter.

Le C-index mesure la capacité de discrimination du classement des risques dans un modèle de survie. Priorité à Uno’s C pour ajuster la censure ; fallback à Harrell’s si nécessaire.

Objectif C-index
Mesure le ratio de paires de patients concordantes : risque élevé → décès précoce.  
Pour i, j : comparable si $T_i < T_j$ et $\delta_i = 1$.  
Concordante si $r_i > r_j$.  
$$
C = \frac{\text{Nombre de paires concordantes}}{\text{Nombre de paires valides}}
$$
- C=1 : parfait ; C=0.5 : aléatoire ; C<0.5 : inversé.

Problème de censure

La censure ($\delta_i=0$) fait perdre des paires de comparaison → biaisé. Solution : Uno’s C (IPCW) utilise des poids inverses.

Uno’s C (IPCW)

Estimer $G(t) = P(C > t)$ par Kaplan–Meier sur cohorte de référence (généralement train). Si train peu d'événements → utiliser test.  
$$
C_{Uno} = \frac{\sum_{i \neq j} I(T_i < T_j) I(\delta_i=1) \frac{I(r_i > r_j)}{G(T_i)^2}}{\sum_{i \neq j} I(T_i < T_j) I(\delta_i=1) \frac{1}{G(T_i)^2}}
$$
Poids $1/G(T_i)^2$ réduit le biais.

Pratique (sksurv)

```python
from sksurv.metrics import concordance_index_ipcw
y_train = Surv.from_arrays(event=train_df[event_col].astype(bool), time=train_df[duration_col])
y_test = Surv.from_arrays(event=test_df[event_col].astype(bool), time=test_df[duration_col])
c_uno, _ = concordance_index_ipcw(y_train, y_test, -risk_score, test_df[duration_col])
```

Fallback : Harrell’s C

Si test sans événements/erreur IPCW :  
$$
C_{Harrell} = \frac{\sum_{i<j} I(T_i < T_j, \delta_i=1) I(r_i > r_j)}{\sum_{i<j} I(T_i < T_j, \delta_i=1)}
$$
```python
from sksurv.metrics import concordance_index_censored
c_harrell, _, _ = concordance_index_censored(events, times, -risk_score)
```

Nettoyage des entrées

| Vérification | Raison | Traitement |
|--------------|--------|------------|
| NaN/Inf dans T/δ/r | Erreur de formule | Filtrer `mask_valid = np.isfinite(...)` |
| Risque sans variance | C insignifiant | Noter “risque constant” |
| Tous événements=0/1 | Pas de paires valides | Logger l'erreur |

```python
mask_valid = np.isfinite(times) & np.isfinite(events) & np.isfinite(risk_score)
if np.var(risk_score) == 0: print("⚠️ Pas de variance dans le risque ; C insignifiant.")
```

Notes de rapport

Noter type C, cohorte de référence, événements test.  
Exemple : `[uno] C=0.712 (ref: train; events:40) | [harrell-fallback] C=0.693`

Interprétation

| C-index | Évaluation | Interprétation |
|---------|------------|----------------|
| <0.55 | Proche aléatoire | Pas de discrimination |
| 0.55–0.65 | Faible–Moyenne | Signal faible |
| 0.65–0.75 | Bonne | Discrimination haut/bas |
| 0.75–0.85 | Très bonne | Fiable |
| >0.85 | Trop bonne | Vérifier surapprentissage |

**Conclusion** : Uno’s C (IPCW) est le plus standard pour la censure, utilise des poids pour corriger le biais. Fallback à Harrell’s si non feasible. Nettoyage des entrées assure la signification ; combiner pour évaluation cohérente sur données censurées.

In [ ]:
# ---------- 2) C-index helper ----------
def safe_cindex(times, events, preds, ref_times, ref_events, prefer="uno"):
    """
    Trả về (c_index, method_note).
    - Lọc NaN/Inf và căn chỉnh kích thước
    - Nếu test không có event -> Harrell
    - Nếu IPCW lỗi vì bất kỳ lý do gì -> fallback Harrell
    - Nếu vẫn không thể, trả về (np.nan, 'skip')
    """
    # ép về 1D
    times  = np.asarray(times).reshape(-1)
    events = np.asarray(events).reshape(-1).astype(int)
    preds  = np.asarray(preds).reshape(-1)

    # sanity mask test
    m_test = np.isfinite(times) & np.isfinite(events) & np.isfinite(preds)
    times2, events2, preds2 = times[m_test], events[m_test], preds[m_test]

    # không đủ mẫu để tính
    if times2.size < 2 or preds2.size < 2:
        return np.nan, "skip(test<2)"

    # nếu toàn cùng 1 giá trị dự báo -> c-index vô nghĩa
    if np.nanstd(preds2) == 0:
        # vẫn có thể trả Harrell (sẽ ~0.5 nếu ngẫu nhiên)
        try:
            res = concordance_index_censored(events2.astype(bool), times2, -preds2)
            return float(res[0]), "harrell(constant-score)"
        except Exception:
            return np.nan, "skip(constant-score)"

    # cohort tham chiếu để ước G(t); nếu train không đủ, dùng test
    ref_times  = np.asarray(ref_times).reshape(-1)
    ref_events = np.asarray(ref_events).reshape(-1).astype(int)
    m_ref = np.isfinite(ref_times) & np.isfinite(ref_events)
    ref_times2, ref_events2 = ref_times[m_ref], ref_events[m_ref]

    y_test = Surv.from_arrays(event=events2.astype(bool), time=times2.astype(float))
    if (ref_times2.size >= 2) and (ref_events2.sum() > 0):
        y_ref = Surv.from_arrays(event=ref_events2.astype(bool), time=ref_times2.astype(float))
    else:
        y_ref = y_test  # fallback: dùng test để ước censoring

    # không có event ở test -> Harrell
    if events2.sum() == 0:
        try:
            res = concordance_index_censored(events2.astype(bool), times2, -preds2)
            return float(res[0]), "harrell(no-event-test)"
        except Exception:
            return np.nan, "skip(no-event-test)"

    # Ưu tiên Uno’s C; nếu lỗi thì fallback Harrell
    if prefer == "uno":
        try:
            c, _ = concordance_index_ipcw(y_ref, y_test, -preds2, times2)
            return float(c), "uno"
        except Exception as e:
            # fallback: Harrell
            try:
                res = concordance_index_censored(events2.astype(bool), times2, -preds2)
                return float(res[0]), f"harrell(fallback:{type(e).__name__})"
            except Exception:
                return np.nan, "skip(fallback-failed)"
    else:
        # trực tiếp Harrell
        try:
            res = concordance_index_censored(events2.astype(bool), times2, -preds2)
            return float(res[0]), "harrell"
        except Exception:
            return np.nan, "skip(harrell-failed)"

# ---------- 3) Evaluate ----------
times, events = test_df[duration_col].values, test_df[event_col].values
ref_times, ref_events = train_df[duration_col].values, train_df[event_col].values

print(
    "Sanity before eval |",
    "len(times)=", len(times),
    "len(events)=", len(events),
    "len(risk_cox)=", len(risk_cox),
    "NaN risk_cox=", np.isnan(risk_cox).sum(),
    "Inf risk_cox=", np.isinf(risk_cox).sum()
)


une étape de vérification rapide (« sanity check ») avant d'évaluer l'indice C dans un modèle de survie (survival).  
Elle vous aide à confirmer que les données d'entrée pour la partie évaluation du modèle sont valides, sans manquants ni erreurs.

In [ ]:

c_cox, m1 = safe_cindex(times, events, risk_cox, ref_times, ref_events, prefer="uno")
c_aft, m2 = safe_cindex(times, events, risk_aft, ref_times, ref_events, prefer="uno")
print(f"[{m1}] C-index Cox = {c_cox:.3f} | [{m2}] C-index AFT = {c_aft:.3f}")


# 8) Stratification des risques et visualisation

**Description courte** :  
* **Kaplan–Meier par décile de risque** : Diviser le test en 10 groupes par score de risque (Cox) ; tracer les courbes de survie pour chaque groupe. Séparation claire (groupe élevé descend rapidement) → modèle discriminatif bon.  
* (Optionnel) **KM par mutation** : Comparer avec/sans mutation (TP53, NPM1…) ; avec test log-rank.

La stratification des risques par Kaplan–Meier (KM) selon les déciles de score de risque est une façon de visualiser la capacité discriminante du modèle Cox/AFT.

## Objectif
Visualiser la discrimination des risques : risque élevé → courbe de survie descend rapidement ; risque faible → courbe de survie élevée longtemps. Écart clair → modèle bon pour la clinique.

## Calcul du score de risque
- **Cox** : $r_i = \hat{\beta}^\top X_i$ (ou `predict_partial_hazard`).  
- **AFT** : $r_i = -\hat{T}_{\text{median},i}$ (temps court → risque élevé).

## Stratification par décile
Diviser le test en 10 groupes selon $r_i$ :  
$$
G_k = \{ i : q_{k-1} < r_i \le q_k \}, \quad k=1\dots10
$$  
(q_k : quantile k/10). Groupe 1 : faible ; 10 : élevé.  
```python
test_df["risk_group"] = pd.qcut(risk_cox, q=10, labels=False) + 1
```

## Estimateur Kaplan–Meier
Fonction de survie pour le groupe k :  
$$
\hat{S}_k(t) = \prod_{t_i \le t} \left(1 - \frac{d_i^{(k)}}{n_i^{(k)}}\right)
$$  
- $d_i^{(k)}$ : décès à t_i ; $n_i^{(k)}$ : survivants.  
Hasard élevé → $\hat{S}_k(t)$ descend rapidement.

## Pratique (lifelines)
```python
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt

kmf = KaplanMeierFitter()
plt.figure(figsize=(7,5))
for g in sorted(test_df["risk_group"].unique()):
    mask = test_df["risk_group"] == g
    kmf.fit(test_df.loc[mask, "OS_YEARS"], 
            event_observed=test_df.loc[mask, "OS_STATUS_BIN"], 
            label=f"Decile {g}")
    kmf.plot_survival_function(ci_show=False)
plt.title("KM Curves by Risk Decile (Cox)")
plt.xlabel("Time (years)"); plt.ylabel("Survival probability")
plt.legend(title="Risk group (1=Low → 10=High)"); plt.grid(alpha=0.3)
plt.show()
```

## Interprétation
| Phénomène | Interprétation |
|-----------|----------------|
| Groupe 10 descend rapidement | Prédiction correcte high risk |
| Groupe 1 stable | Pronostic bon |
| Courbes séparées, non croisées | Stratification bonne |
| Courbes superposées | Pas de discrimination |
| Fluctuations fortes | Échantillon petit → variabilité |

## Test log-rank
Confirmer les différences : $H_0: S_1(t) = \dots = S_{10}(t)$.  
```python
from lifelines.statistics import multivariate_logrank_test
results = multivariate_logrank_test(test_df["OS_YEARS"], test_df["risk_group"], 
                                    event_observed=test_df["OS_STATUS_BIN"])
results.print_summary()  # p < 0.05 → différence significative
```

## Variantes
- Tertile/quartile (3–4 groupes) si données peu nombreuses.  
- KM par mutation (optionnel) : Comparer avec/sans TP53/NPM1 ; test log-rank.  
  ```python
  kmf.fit(df[mut=="Yes"], "OS_YEARS", event_observed=..., label="TP53 mut")
  kmf.plot_survival_function()
  # Log-rank : logrank_test entre 2 groupes
  ```

## Exemple de résultats
| Groupe | Médiane de survie (années) | p-value |
|--------|----------------------------|---------|
| Décile 1 (faible) | 5.8 | |
| Décile 10 (élevé) | 0.9 | <1e-6 |

Graphique : Groupe 10 (rouge) descend <1 an ; groupe 1 (bleu) >5 ans → séparation forte.

**Conclusion** : KM par décile de score de risque illustre la discrimination des risques ; courbes séparées + log-rank bas → modèle fort, interprétable cliniquement. Combiner avec C-index pour évaluation complète.

In [ ]:
test_plot = test_df.copy()
test_plot["risk_cox"] = risk_cox
# trong trường hợp nhiều giá trị trùng, dùng duplicates="drop"
test_plot["decile"] = pd.qcut(test_plot["risk_cox"], q=10, labels=False, duplicates="drop")

plt.figure(figsize=(8,6))
km = KaplanMeierFitter()
for d in sorted(test_plot["decile"].unique()):
    grp = test_plot[test_plot["decile"] == d]
    if len(grp) < 5:  # nhóm quá ít thì bỏ qua
        continue
    km.fit(grp[duration_col].values, grp[event_col].values, label=f"Decile {int(d)+1}")
    km.plot_survival_function(ci_show=False)

plt.title("Kaplan–Meier by risk decile (Cox)")
plt.xlabel("Years"); plt.ylabel("Survival probability")
plt.legend(ncol=2)
plt.tight_layout()
plt.show()


# 9) Calibration des probabilités à des points temporels (calibration)

**Description courte** :  
* Objectif : Estimer $\hat{p}(t_0 \mid X) = P(T > t_0 \mid X)$ à des points temporels cliniques (1 an, 2 ans, 3 ans…).  
* **Construire variable binaire** : y=1 si survie au-delà de t₀ (ou censure après t₀) ; exclure les censures avant t₀.  
* **Calibration** : Platt (sigmoïde logistique) pour biais en S, simple ; Isotonique (non paramétrique) flexible mais sujet à surapprentissage si données peu nombreuses.  
* **Évaluation** : Tracer **diagramme de fiabilité** (ligne proche de la diagonale 45° → bon).  
* Recommandation : Séparer un « ensemble de calibration » ou utiliser validation croisée pour éviter surapprentissage.

La calibration convertit le score de risque en probabilité de survie réelle, assurant predicted ≈ observed pour les prévisions cliniques.

## Objectif
Prédire la probabilité de survie au-delà de t₀ :  
$$
\hat{p}(t_0 \mid X) = \widehat{P}(T > t_0 \mid X)
$$  
Aligner le score de risque sur la fréquence de survie observée.

## Préparation des données
À t₀ (ex. : 1 an) :  
$$
y_i = 
\begin{cases} 
1 & T_i > t_0 \ (survie/censure après) \\ 
0 & T_i \le t_0, \ \delta_i=1 \ (décès avant) \\ 
\text{exclure} & T_i \le t_0, \ \delta_i=0 \ (censure avant)
\end{cases}
$$  
→ $\mathcal{D}_{cal} = \{(r_i, y_i)\}$, r_i = risque de Cox/AFT.

## Méthodes de calibration
Trouver $f: r_i \mapsto \hat{p}_i = P(T > t_0 \mid X_i)$.

### Platt (sigmoïde logistique)
Pour biais en S (trop optimiste/pessimiste aux extrémités) :  
$$
P(T > t_0 \mid X) = \frac{1}{1 + \exp(-(a + b r))}
$$  
Ajuster logistique sur $\mathcal{D}_{cal}$.  
Avantage : Simple, peu de surapprentissage. Inconvénient : Peu flexible pour non linéarités.

### Régression isotonique
Non paramétrique, préservant la monotonicité :  
$$
f^* = \arg\min_f \sum_i (f(r_i) - y_i)^2 \ \text{s.t.} \ f(r_i) \le f(r_j) \ \text{si} \ r_i < r_j
$$  
Avantage : Flexible. Inconvénient : Surapprentissage si peu de données (<100 événements).

## Évaluation : Diagramme de fiabilité
Diviser en 10 groupes selon $\hat{p}_i$ :  
| Notation | Signification |  
|----------|---------------|  
| $\bar{p}_j = mean(\hat{p}_i)$ | Prédiction moyenne groupe j |  
| $\bar{y}_j = mean(y_i)$ | Observation réelle groupe j |  

Tracer $\bar{y}_j$ vs. $\bar{p}_j$ :  
- Diagonale 45° : Parfait.  
- Sous diagonale : Trop optimiste (sur-estimation survie).  
- Au-dessus diagonale : Trop pessimiste (sous-estimation).

## Pratique (sklearn/lifelines)
```python
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import calibration_curve
import numpy as np; import matplotlib.pyplot as plt

# 1. Construire y_bin à t0=1.0
mask_keep = (times > t0) | ((times <= t0) & (events == 1))
y_bin = (times > t0).astype(int)[mask_keep]
risk = risk_cox[mask_keep]

# 2. Calibration
platt = LogisticRegression().fit(risk.reshape(-1,1), y_bin)
p_platt = platt.predict_proba(risk.reshape(-1,1))[:,1]

iso = IsotonicRegression(out_of_bounds='clip').fit(risk, y_bin)
p_iso = iso.predict(risk)

# 3. Graphique de fiabilité
plt.figure(figsize=(5,5)); plt.plot([0,1],[0,1],'--',color='gray')
for name, p_pred in [("Platt", p_platt), ("Isotonic", p_iso)]:
    fraction_of_positives, mean_predicted_value = calibration_curve(y_bin, p_pred, n_bins=10)
    plt.plot(mean_predicted_value, fraction_of_positives, 'o-', label=name)
plt.xlabel("Predicted P(T > t₀)"); plt.ylabel("Observed frequency")
plt.legend(); plt.title("Calibration at 1 year"); plt.show()
```

## Évaluation quantitative (optionnelle)
Score de Brier à t₀ :  
$$
\text{Brier}(t_0) = \frac{1}{N} \sum_i (I(T_i > t_0) - \hat{p}_i)^2
$$  
Faible → bon. IBS : Moyenne de Brier sur le temps.

## Procédure pratique
| Étape | Objectif | Réalisation |
|-------|----------|-------------|
| 1. Choisir t₀ (1 an,2 ans,3 ans) | Adapté clinique | Basé sur étude |
| 2. Créer y binaire | Problème binaire | Exclure censurés avant t₀ |
| 3. Calibrer | Éliminer biais | Platt/Isotonique sur $\mathcal{D}_{cal}$ |
| 4. Évaluer | Vérifier alignement | Fiabilité + Brier |
| 5. Éviter surapprentissage | Réfléchir hors échantillon | Ensemble calibration séparé/CV |

## Pratique & recommandations
| Situation | Méthode | Raison |
|-----------|---------|--------|
| Données ≥500 | Isotonique | Flexible non linéaire |
| Données petites/peu d'événements | Platt | Stable, peu de surapprentissage |
| Déploiement réel | Platt/spline sur hold-out | Facile à stocker/appliquer |
| CV | `calibration_curve` par fold | Éviter chevauchement entraînement |

**Conclusion** : Calibration à t₀ convertit risque en probabilité de survie réelle (y=1 si au-delà du point). Platt/Isotonique élimine biais ; diagramme de fiabilité vérifie (proche 45° → calibré). Utiliser ensemble séparé/CV évite surapprentissage → modèle de prédiction personnalisée fiable pour clinique.

In [ ]:
def make_binary_at_t0(time, event, t0):
    # y=1 nếu T>t0 (sống qua mốc) hoặc kiểm duyệt sau t0
    y = ((time > t0) | ((time <= t0) & (event==0))).astype(int)
    # loại các mẫu kiểm duyệt trước t0 (không biết kết cục tại t0)
    mask = ~((time < t0) & (event==0))
    return y[mask], mask

def reliability(y_true, y_pred, bins=np.linspace(0,1,10)):
    d = pd.DataFrame({"y":y_true, "p":y_pred})
    d["bin"] = pd.cut(d["p"], bins)
    return d.groupby("bin")[["y","p"]].mean().dropna()

for t0 in [1.0, 2.0, 3.0]:
    y_calib, mask = make_binary_at_t0(times, events, t0)
    sc = risk_cox[mask]
    # bỏ qua nếu chỉ có 1 lớp (toàn sống/toàn sự kiện)
    if len(np.unique(y_calib)) < 2 or len(y_calib) < 30:
        print(f"[Calibration t0={t0}] Bỏ qua (ít điểm hoặc chỉ một lớp).")
        continue

    # Platt
    pl = LogisticRegression(max_iter=1000).fit(sc.reshape(-1,1), y_calib)
    p_platt = pl.predict_proba(sc.reshape(-1,1))[:,1]

    # Isotonic
    iso = IsotonicRegression(out_of_bounds="clip").fit(sc, y_calib)
    p_iso = iso.transform(sc)

    r_pl  = reliability(y_calib, p_platt)
    r_iso = reliability(y_calib, p_iso)

    # vẽ
    plt.figure(figsize=(5,5))
    plt.plot([0,1],[0,1],'--',color='gray')
    plt.plot(r_pl["p"], r_pl["y"], 'o-', label="Platt")
    plt.plot(r_iso["p"], r_iso["y"], 's-', label="Isotonic")
    plt.xlabel(f"Predicted P(T>{t0}y)")
    plt.ylabel("Observed frequency")
    plt.title(f"Calibration at {t0} years (Cox risk)")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # lưu calibration points
    # r_pl.assign(t0=t0, method="platt").to_csv(f"./data/calib_points_platt_t{int(t0)}.csv")
    # r_iso.assign(t0=t0, method="isotonic").to_csv(f"./data/calib_points_isotonic_t{int(t0)}.csv")

print("Đã lưu calibration points (nếu có): /mnt/data/calib_points_*")


# 10) Rapport et export des résultats

* **Coefficients et signification Cox** : tableau de coefficients, HR = exp(coef), IC, p-value ; sauvegarder le tableau résumé.
* **Paramètres AFT** : estimations et signification.
* **C-index** (Uno/Harrell) pour Cox et AFT ; préciser la méthode (Uno/Harrell) et la raison du fallback.
* **Graphiques** :

  * KM par décile (Cox), KM par grandes mutations.
  * Calibration à t₀ (1 an, 2 ans, 3 ans).
* **Prédictions test** :

  * Sauvegarder le score de risque sur test, et si nécessaire, la probabilité de survie à t₀.
  * (Avec Cox) utiliser la survie de base pour déduire ( \hat S(t|x) = \hat S_0(t)^{\exp(\beta^\top x)} ) et extraire spécifiquement à t₀.

In [ ]:
# =====================
# 4) PREDICT SURVIVAL AT t0 (Cox)
# =====================
# Lifelines allows computing S_hat(t|x) from baseline + partial hazard.
# For brevity, we directly compute the probabilities at t0 for the test set.

t0_list = [1.0, 2.0, 3.0]

# baseline survival according to Cox
baseline_surv = cph.baseline_survival_  # index is time (same unit as OS_YEARS)
# Utility function: interpolate S0(t) then exponentiate according to exp(beta^T x)
def predict_survival_prob_at_t0(cox_model, X, t0):
    # get S0(t0) by nearest interpolation (forward fill)
    S0 = float(baseline_surv.reindex(baseline_surv.index.union([t0])).sort_index().ffill().loc[t0].values)
    # partial hazard = exp(beta^T x) → S(t|x) = S0(t) ** exp(beta^T x)
    ph = cox_model.predict_partial_hazard(X).values.ravel()
    return (S0 ** ph)

pred_out = test_df[["OS_YEARS","OS_STATUS"]].copy()
for t0 in t0_list:
    pred_out[f"P_surv_gt_{int(t0)}y_cox"] = predict_survival_prob_at_t0(cph, test_df[Xcols], t0)

pred_out["risk_cox"] = risk_cox
pred_out
# pred_out.to_csv("./data/test_predictions_cox.csv", index=False)
# print("Saved: /mnt/data/test_predictions_cox.csv")

# 11) Diagnostic et résolution des problèmes courants

* **Matrice singulière / non convergence (Cox)** :

  * Déjà log/clip/contrôle des outliers, suppression de variance proche de zéro, réduction de colinéarité (corrélation élevée, VIF), ajout de **pénaliseur ridge**.
  * Si toujours difficile : essayer **stratification** selon la variable violant PH, ou passer à **AFT**.
* **Test sans événements** :

  * Ajuster la méthode de division (stratification + contrainte sur le nombre d'événements), ou ajouter une procédure « transfert de quelques cas d'événements » de train vers test.
  * Si le dataset n'a vraiment pas d'événements (tous vivants/censurés) : **impossible** d'évaluer la discrimination par C-index ; rediriger vers description/seulement censure.
* **Erreur IPCW due à tableau vide** :

  * Toujours filtrer NaN/Inf avant ; si après filtrage trop peu d'observations (<2) alors noter « skip ».
* **Scores prédits sans variance** :

  * Revoir le pipeline de prétraitement ; vérifier s'il y a une étape rendant tous les scores identiques (ex. : scale erroné, suppression de colonnes, logique de risque constante).

In [ ]:
# =====================
# 5) EXPORT EXTRAS
# =====================
# Importance proxy: |coef| (Cox)
imp = cph.params_.abs().sort_values(ascending=False).rename("abs_coef")
# imp.to_csv("./data/cox_importance_abscoef.csv")
# print("Đã lưu: /mnt/data/cox_importance_abscoef.csv")

# Bảng tóm tắt kết quả chính
summary = pd.DataFrame({
    "metric": ["C-index (Cox)", "C-index (AFT)"],
    "value":  [c_cox, c_aft],
    "estimator": [m1, m2]
})
# summary.to_csv("./data/summary_metrics.csv", index=False)
summary


# 12) Validation et tests

* **Validation croisée stratifiée par événement** (k-fold) et **CV groupée par CENTER** (si centres/hôpitaux présents) pour vérifier la généralisabilité.
* **Analyse de sensibilité** : changements légers sur winsorize/pénaliseur/transformation log, vérifier si C-index et calibration stables.
* **Explication du modèle** :

  * Cox : basée sur HR, test PH.
  * AFT : coefficients interprétés selon log-temps.
  * (Optionnel) SHAP/Permutation pour modèles non linéaires (si utilisation de GBM/NN plus tard).


# 13) Gouvernance du modèle et reproductibilité

* **Enregistrer les versions** : données, liste des caractéristiques finales, paramètres du modèle, pénaliseur, t₀ calibration.
* **Sauvegarder le pipeline** : diagramme ETL → Caractéristiques → Division → Entraînement → Évaluation → Rapport.
* **Contrôle qualité** : checklist avant entraînement (taux d'événements, nombre de NA, corr/VIF, échelles).
* **Reproductibilité des résultats** : sauvegarder tous les artefacts (tableau de coefficients, C-index, graphiques KM, points de calibration, fichier de prédictions test).


## Critères « travail terminé »

* Nombre total d'**événements > 0** et **événements présents dans test**.
* C-index (Uno ou fallback Harrell) calculé avec succès pour **Cox** et **AFT**.
* Graphique **KM par décile** montrant une stratification des risques raisonnable.
* **Calibration** à t₀ exécutée et ligne de fiabilité proche de 45°.
* Rapport complet exporté : coefficients Cox (HR, IC, p), paramètres AFT, tableau C-index, fichier de prédictions test, figures KM, figures de calibration.